In [186]:
import pandas as pd
import numpy as np
import os
import sys
import json

In [187]:
from sklearn.preprocessing import StandardScaler

In [188]:
import torch
from scipy.ndimage import median_filter
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch import optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [189]:
from pickle import load
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt


In [190]:
from sklearn.metrics import silhouette_score

In [191]:
data = pd.read_csv('../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/features_v2.csv')
feature_cols = [c for c in data.columns if "_" in c]
for col in feature_cols:
    data[col] = data[col].clip(data[col].mean() - (3*data[col].std()), data[col].mean() + (3*data[col].std()))

In [192]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super().__init__()
        self.l1 = nn.Linear(input_dim, 128)
        
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 128)
        self.l4 = nn.Linear(128, embed_dim)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.dropout(x)
        x = self.relu(self.l2(x))
        x = self.dropout(x)
        x = self.relu(self.l3(x))
        x = self.dropout(x)
        x = self.l4(x)
        return x


class ClassificationHead(nn.Module):
    def __init__(self,embed_dim, output_dim):
        super().__init__()
        self.l1 = nn.Linear(embed_dim, 64)
        self.l2 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.l1(x))
        x = self.l2(x)
        return x

class UserClassifier(nn.Module):
    def __init__(self, input_dim, embed_dim, output_dim):
        super().__init__()
        self.encoder = Encoder(input_dim, embed_dim)
        self.decoder = ClassificationHead(embed_dim, output_dim)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [193]:
train_parts = list(range(1,39))
test_parts = list(range(39,44))

In [194]:
train_df = data[data["participant"].isin(train_parts)].reset_index()
test_df = data[data["participant"].isin(test_parts)].reset_index()

In [195]:
scaler  = load(open('sub_scaler.pkl', 'rb'))

In [196]:
x_train = train_df.loc[:, feature_cols].values
y_train = (train_df.loc[:, 'participant']).values
g_train = (train_df.loc[:, 'gesture']).values
s_train = (train_df.loc[:, 'session']).values

x_test = test_df.loc[:, feature_cols].values
y_test = (test_df.loc[:, 'participant']).values
g_test = (test_df.loc[:, 'gesture']).values
s_test = (test_df.loc[:, 'session']).values

In [197]:
all_y = np.concatenate([y_train, y_test])
all_g = np.concatenate([g_train, g_test])
all_s = np.concatenate([s_train, s_test])

In [198]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [199]:
encoder = Encoder(x_train.shape[1], 16)

In [200]:
encoder.load_state_dict(torch.load('sub_encoder.pth'), strict=False)

<All keys matched successfully>

In [201]:
def embed(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            # _, predicted = torch.max(outputs, 1)
            y_pred.extend(outputs)

    return torch.stack(y_pred)

In [202]:
## Reading centroids
with open("centroids_v1.json", "r") as file:
    centroids = json.load(file)

In [203]:
## Reading centroids
with open("test_centroids_v1.json", "r") as file:
    test_centroids = json.load(file)

In [204]:
all_centroids = {}
for k in centroids.keys():
    all_centroids[k] = {}
    all_centroids[k].update(centroids[k])
    all_centroids[k].update(test_centroids[k])

In [205]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [206]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [207]:
train_embeddings = embed(model=encoder, test_loader=train_loader)
test_embeddings = embed(model=encoder, test_loader=test_loader)

In [208]:
all_embeddings = np.concatenate([train_embeddings, test_embeddings], axis = 0)

In [209]:
all_embeddings.shape

(15351, 16)

In [210]:
centroids.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17'])

In [211]:
gestures = range(1,18)

In [87]:
def predict_cluster(centroids, embedding, gesture, parts):
    cents = centroids[str(gesture)]
    min_dist = 1e10
    min_part = None
    for i in parts:
        c = cents[str(i)]['centroid']
        dist = np.linalg.norm(embedding - c)
        if dist< min_dist:
            min_dist = dist
            min_part = i
    return min_part
    
    

In [94]:
i = np.where(g_train==10)
embeds = train_embeddings[i]
labels = y_train[i]

In [95]:
(embeds[10,:]).shape

torch.Size([16])

In [96]:
preds = []
for e in tqdm(range(len(embeds))):
    embedding = np.array(embeds[e,:])
    # print(embedding.shape)
    pred = predict_cluster(centroids, np.array(embedding), 10, train_parts)
    preds.append(pred)

100%|██████████████████████████████████████████████████████████████████████████████| 798/798 [00:00<00:00, 2247.91it/s]


In [97]:
accuracy_score(preds, labels)

0.9223057644110275

In [98]:
g_accs = {}
for g in tqdm(gestures):
    i = np.where(g_train==g)
    embeds = train_embeddings[i]
    labels = y_train[i]
    preds = []
    for e in tqdm(range(len(embeds))):
        embedding = np.array(embeds[e,:])
        # print(embedding.shape)
        pred = predict_cluster(centroids, np.array(embedding), g, train_parts)
        preds.append(pred)
    g_accs[g] = accuracy_score(preds, labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:08<00:00,  2.10it/s]


In [99]:
g_accs_test = {}
for g in tqdm(gestures):
    i = np.where(g_test==g)
    embeds = test_embeddings[i]
    labels = y_test[i]
    preds = []
    for e in tqdm(range(len(embeds))):
        embedding = np.array(embeds[e,:])
        # print(embedding.shape)
        pred = predict_cluster(test_centroids, np.array(embedding), g, test_parts)
        preds.append(pred)
    g_accs_test[g] = accuracy_score(preds, labels)

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 6562.29it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 4999.51it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 6563.46it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 8077.80it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 8755.33it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 8077.66it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 8073.21it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 8069.81it/s]

100%|███████████████████████████

In [245]:
sorted(g_accs.items(), key = lambda x: x[1], reverse=True)

[(2, 0.9761904761904762),
 (7, 0.9649122807017544),
 (3, 0.9611528822055138),
 (14, 0.9611528822055138),
 (4, 0.9598997493734336),
 (5, 0.9598997493734336),
 (6, 0.9598997493734336),
 (9, 0.9573934837092731),
 (15, 0.9548872180451128),
 (13, 0.9486215538847118),
 (16, 0.9423558897243107),
 (8, 0.9398496240601504),
 (11, 0.9385964912280702),
 (12, 0.9335839598997494),
 (10, 0.9223057644110275),
 (1, 0.899749373433584),
 (17, 0.7268170426065163)]

In [246]:
sorted(g_accs_test.items(), key = lambda x: x[1], reverse=True)

[(6, 0.9238095238095239),
 (10, 0.9047619047619048),
 (2, 0.8952380952380953),
 (11, 0.8952380952380953),
 (13, 0.8857142857142857),
 (3, 0.8761904761904762),
 (5, 0.8761904761904762),
 (15, 0.8666666666666667),
 (9, 0.8571428571428571),
 (4, 0.8380952380952381),
 (8, 0.8380952380952381),
 (1, 0.8285714285714286),
 (16, 0.8285714285714286),
 (14, 0.8095238095238095),
 (7, 0.7428571428571429),
 (12, 0.6952380952380952),
 (17, 0.580952380952381)]


## Bucket based Authentication

### Train

In [134]:
auth_set = set(np.random.choice(range(1, 38), size = 5, replace=False))

In [135]:
auth_set

{1, 3, 10, 29, 36}

In [136]:
def auth(emb:np.ndarray, auth_set:set, centroids, gesture_val, parts)-> bool:
    pred = predict_cluster(centroids, np.array(emb), gesture_val, parts)
    # print(pred)
    if pred in auth_set:
        return 1
    return 0
    # pass

In [137]:
g = 2

In [138]:
auths = []
i = np.where(g_train==g)
embeds = train_embeddings[i]
labels = y_train[i]
for e in tqdm(range(len(embeds))):
    embedding = np.array(embeds[e,:])
    auth_code = auth(embedding, auth_set, centroids, g, train_parts)
    auths.append(auth_code)

100%|██████████████████████████████████████████████████████████████████████████████| 798/798 [00:00<00:00, 1805.45it/s]


In [139]:
actual_auth = [1 if x in auth_set else 0 for x in labels]

In [140]:
def far(true, preds):
    true = np.array(true)
    preds = np.array(preds)
    far = np.sum((true==0)&(preds==1))
    return far / (len(true) - np.sum(true))

In [241]:
def frr(true, preds):
    true = np.array(true)
    preds = np.array(preds)
    frr = np.sum((true==1)&(preds==0))
    # print(frr)
    return frr / np.sum(true)

In [142]:
far(actual_auth, auths)

0.001443001443001443

In [143]:
frr(actual_auth, auths)

0.0380952380952381

In [144]:
accuracy_score(actual_auth, auths)

0.993734335839599

In [160]:
num_exp = 50

In [161]:
aggregates_5_auth = {g:0 for g in gestures}
for g in gestures:
    aggregates_5_auth[g] = {}
    aggregates_5_auth[g]['far'] = 0
    aggregates_5_auth[g]['frr'] = 0
    aggregates_5_auth[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 38), size = 5, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_train==g)
        embeds = train_embeddings[i]
        labels = y_train[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, centroids, g, train_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_5_auth[g]['far'] += far(actual_auth, auths)
        aggregates_5_auth[g]['frr'] += frr(actual_auth, auths)
        aggregates_5_auth[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_5_auth:
    aggregates_5_auth[k]['far'] /= num_exp
    aggregates_5_auth[k]['frr'] /= num_exp
    aggregates_5_auth[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:47<00:00,  6.95s/it]


In [213]:
aggregates_1_auth = {g:0 for g in gestures}
g17 = []
for g in gestures:
    aggregates_1_auth[g] = {}
    aggregates_1_auth[g]['far'] = 0
    aggregates_1_auth[g]['frr'] = 0
    aggregates_1_auth[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 38), size = 1, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_train==g)
        embeds = train_embeddings[i]
        labels = y_train[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, centroids, g, train_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_1_auth[g]['far'] += far(actual_auth, auths)
        aggregates_1_auth[g]['frr'] += frr(actual_auth, auths)
        aggregates_1_auth[g]['accuracy'] += accuracy_score(actual_auth, auths)
        if g == 17:
            g17.append((actual_auth, auths))
for k in aggregates_5_auth:
    aggregates_1_auth[k]['far'] /= num_exp
    aggregates_1_auth[k]['frr'] /= num_exp
    aggregates_1_auth[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:15<00:00,  6.31s/it]


In [215]:
print(len(g17))

50


In [230]:
frr(g17[0][0], g17[0][1])

8


0.38095238095238093

In [231]:
np.sum((np.array(g17[0][0])==1)&(np.array(g17[0][1])==0)) / np.sum(g17[0][0])

0.38095238095238093

In [238]:
precision_score(np.array(g17[0][0]), np.array(g17[0][1]))

0.5652173913043478

In [236]:
g17[0][1]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [235]:
g17[0][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [219]:
sum(g17[0][1])

23

In [220]:
21/23

0.9130434782608695

In [163]:
aggregates_10_auth = {g:0 for g in gestures}
for g in gestures:
    aggregates_10_auth[g] = {}
    aggregates_10_auth[g]['far'] = 0
    aggregates_10_auth[g]['frr'] = 0
    aggregates_10_auth[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 38), size = 10, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_train==g)
        embeds = train_embeddings[i]
        labels = y_train[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, centroids, g, train_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_10_auth[g]['far'] += far(actual_auth, auths)
        aggregates_10_auth[g]['frr'] += frr(actual_auth, auths)
        aggregates_10_auth[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_5_auth:
    aggregates_10_auth[k]['far'] /= num_exp
    aggregates_10_auth[k]['frr'] /= num_exp
    aggregates_10_auth[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:45<00:00,  6.90s/it]


In [168]:
aggregates_20_auth = {g:0 for g in gestures}
for g in gestures:
    aggregates_20_auth[g] = {}
    aggregates_20_auth[g]['far'] = 0
    aggregates_20_auth[g]['frr'] = 0
    aggregates_20_auth[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 38), size = 20, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_train==g)
        embeds = train_embeddings[i]
        labels = y_train[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, centroids, g, train_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_20_auth[g]['far'] += far(actual_auth, auths)
        aggregates_20_auth[g]['frr'] += frr(actual_auth, auths)
        aggregates_20_auth[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_20_auth:
    aggregates_20_auth[k]['far'] /= num_exp
    aggregates_20_auth[k]['frr'] /= num_exp
    aggregates_20_auth[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [06:16<00:00,  7.52s/it]


In [169]:
pd.DataFrame(aggregates_20_auth).T.sort_values(by = "accuracy", ascending=False)

,far,frr,accuracy
2,0.012116,0.012905,0.987469
7,0.021005,0.016714,0.981253
3,0.016138,0.022476,0.980526
14,0.015820,0.024333,0.979699
6,0.019153,0.021476,0.979624
5,0.024339,0.019238,0.978346
9,0.021376,0.022476,0.978045
4,0.022169,0.021810,0.978020
15,0.021746,0.024810,0.976642
13,0.024921,0.027476,0.973734


In [166]:
pd.DataFrame(aggregates_10_auth).T.sort_values(by = "accuracy", ascending=False)

,far,frr,accuracy
2,0.004864,0.022095,0.990602
14,0.007653,0.032000,0.985940
7,0.009456,0.027333,0.985840
3,0.008265,0.032476,0.985363
5,0.011905,0.026381,0.984286
6,0.011667,0.027619,0.984135
4,0.010544,0.031905,0.983835
9,0.011497,0.033810,0.982632
15,0.012993,0.035714,0.981028
13,0.013810,0.039905,0.979323


In [239]:
pd.DataFrame(aggregates_1_auth).T.sort_values(by = "accuracy", ascending=False)

,far,frr,accuracy
2,0.000798,0.017143,0.998772
6,0.001055,0.028571,0.998221
4,0.001210,0.022857,0.998221
14,0.000952,0.035238,0.998145
9,0.001184,0.037143,0.997870
7,0.001364,0.030476,0.997870
3,0.001416,0.029524,0.997845
5,0.000927,0.047619,0.997845
16,0.000952,0.048571,0.997794
15,0.000978,0.047619,0.997794


### test

In [170]:
num_exp = 5

In [258]:
aggregates_test = {g:0 for g in gestures}
for g in gestures:
    aggregates_test[g] = {}
    aggregates_test[g]['far'] = 0
    aggregates_test[g]['frr'] = 0
    aggregates_test[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(39, 44), size = 2, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_test==g)
        embeds = test_embeddings[i]
        labels = y_test[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, test_centroids, g, test_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_test[g]['far'] += far(actual_auth, auths)
        aggregates_test[g]['frr'] += frr(actual_auth, auths)
        aggregates_test[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_test:
    aggregates_test[k]['far'] /= num_exp
    aggregates_test[k]['frr'] /= num_exp
    aggregates_test[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:07<00:00,  6.25it/s]


In [259]:
pd.DataFrame(aggregates_test).T.sort_values(by = "accuracy", ascending=False)


,far,frr,accuracy
6,0.033333,0.071905,0.951238
10,0.043810,0.078571,0.942286
11,0.065079,0.061429,0.936381
2,0.057460,0.073333,0.936190
13,0.054603,0.080476,0.935048
5,0.061270,0.095238,0.925143
3,0.051111,0.111905,0.924571
15,0.058413,0.103333,0.923619
9,0.066984,0.102857,0.918667
4,0.076825,0.107619,0.910857


In [173]:
num_exp = 10
aggregates_test = {g:0 for g in gestures}
for g in gestures:
    aggregates_test[g] = {}
    aggregates_test[g]['far'] = 0
    aggregates_test[g]['frr'] = 0
    aggregates_test[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(39, 44), size = 1, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_test==g)
        embeds = test_embeddings[i]
        labels = y_test[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, test_centroids, g, test_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_test[g]['far'] += far(actual_auth, auths)
        aggregates_test[g]['frr'] += frr(actual_auth, auths)
        aggregates_test[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_test:
    aggregates_test[k]['far'] /= num_exp
    aggregates_test[k]['frr'] /= num_exp
    aggregates_test[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.15it/s]


In [174]:
pd.DataFrame(aggregates_test).T.sort_values(by = "accuracy", ascending=False)


,far,frr,accuracy
6,0.014286,0.057143,0.977143
11,0.020238,0.076190,0.968571
5,0.017857,0.109524,0.963810
10,0.021429,0.104762,0.961905
3,0.023810,0.114286,0.958095
2,0.029762,0.104762,0.955238
13,0.022619,0.161905,0.949524
9,0.027381,0.152381,0.947619
15,0.025000,0.176190,0.944762
1,0.028571,0.180952,0.940952


In [175]:
num_exp = 20
aggregates_test = {g:0 for g in gestures}
for g in gestures:
    aggregates_test[g] = {}
    aggregates_test[g]['far'] = 0
    aggregates_test[g]['frr'] = 0
    aggregates_test[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(39, 44), size = 3, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(g_test==g)
        embeds = test_embeddings[i]
        labels = y_test[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, test_centroids, g, test_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_test[g]['far'] += far(actual_auth, auths)
        aggregates_test[g]['frr'] += frr(actual_auth, auths)
        aggregates_test[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_test:
    aggregates_test[k]['far'] /= num_exp
    aggregates_test[k]['frr'] /= num_exp
    aggregates_test[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.11it/s]


In [176]:
pd.DataFrame(aggregates_test).T.sort_values(by = "accuracy", ascending=False)


,far,frr,accuracy
10,0.058333,0.036508,0.954762
6,0.050000,0.042857,0.954286
2,0.060714,0.054762,0.942857
13,0.069048,0.055556,0.939048
5,0.072619,0.055556,0.937619
11,0.082143,0.057143,0.932857
15,0.075000,0.066667,0.930000
9,0.084524,0.070635,0.923810
3,0.091667,0.065873,0.923810
8,0.102381,0.082540,0.909524


### All 

In [177]:
all_parts = range(1, 44)

In [178]:
num_exp = 50
aggregates_5 = {g:0 for g in gestures}
for g in gestures:
    aggregates_5[g] = {}
    aggregates_5[g]['far'] = 0
    aggregates_5[g]['frr'] = 0
    aggregates_5[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 44), size = 5, replace=False))
    for g in gestures:
        # aggregates_5_auth[g] = {}
        auths = []
        i = np.where(all_g==g)
        embeds = all_embeddings[i]
        labels = all_y[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, all_centroids, g, all_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_5[g]['far'] += far(actual_auth, auths)
        aggregates_5[g]['frr'] += frr(actual_auth, auths)
        aggregates_5[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_5:
    aggregates_5[k]['far'] /= num_exp
    aggregates_5[k]['frr'] /= num_exp
    aggregates_5[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [06:39<00:00,  8.00s/it]


In [242]:
num_exp = 50
aggregates_1 = {g:0 for g in gestures}
for g in gestures:
    aggregates_1[g] = {}
    aggregates_1[g]['far'] = 0
    aggregates_1[g]['frr'] = 0
    aggregates_1[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 44), size = 5, replace=False))
    for g in gestures:
        # aggregates_1_auth[g] = {}
        auths = []
        i = np.where(all_g==g)
        embeds = all_embeddings[i]
        labels = all_y[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, all_centroids, g, all_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_1[g]['far'] += far(actual_auth, auths)
        aggregates_1[g]['frr'] += frr(actual_auth, auths)
        aggregates_1[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_1:
    aggregates_1[k]['far'] /= num_exp
    aggregates_1[k]['frr'] /= num_exp
    aggregates_1[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:28<00:00,  6.57s/it]


In [179]:
pd.DataFrame(aggregates_5).T.sort_values(by = "accuracy", ascending=False)


,far,frr,accuracy
2,0.006341,0.050286,0.988549
3,0.008997,0.060762,0.984983
15,0.010526,0.078095,0.981617
5,0.011479,0.078286,0.980753
11,0.011178,0.084381,0.980310
6,0.011880,0.084000,0.979734
14,0.012155,0.088571,0.978959
4,0.011930,0.094857,0.978427
7,0.012707,0.094476,0.977785
13,0.012531,0.096190,0.977741


In [243]:
pd.DataFrame(aggregates_1).T.sort_values(by = "accuracy", ascending=False)


,far,frr,accuracy
2,0.006441,0.044000,0.989192
3,0.009323,0.059619,0.984828
5,0.010952,0.068952,0.982303
15,0.010652,0.072952,0.982104
6,0.011529,0.082667,0.980199
14,0.011729,0.083810,0.979889
4,0.011529,0.087048,0.979690
11,0.011554,0.094286,0.978826
7,0.012907,0.090476,0.978073
8,0.012757,0.093524,0.977852


In [180]:
num_exp = 50
aggregates_10 = {g:0 for g in gestures}
for g in gestures:
    aggregates_10[g] = {}
    aggregates_10[g]['far'] = 0
    aggregates_10[g]['frr'] = 0
    aggregates_10[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 44), size = 10, replace=False))
    for g in gestures:
        # aggregates_10_auth[g] = {}
        auths = []
        i = np.where(all_g==g)
        embeds = all_embeddings[i]
        labels = all_y[i]
        # print(len(labels), g)
        for e in range(len(embeds)):
            embedding = np.array(embeds[e,:])
            auth_code = auth(embedding, auth_set, all_centroids, g, all_parts)
            auths.append(auth_code)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_10[g]['far'] += far(actual_auth, auths)
        aggregates_10[g]['frr'] += frr(actual_auth, auths)
        aggregates_10[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_10:
    aggregates_10[k]['far'] /= num_exp
    aggregates_10[k]['frr'] /= num_exp
    aggregates_10[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [06:33<00:00,  7.88s/it]


In [181]:
auth_set = set(np.random.choice(range(1, 38), size = 5, replace=False))

In [182]:
auth_set

{13, 16, 18, 30, 34}

In [183]:
pd.DataFrame(aggregates_10).T.sort_values(by = "accuracy", ascending=False)


,far,frr,accuracy
2,0.014372,0.038762,0.979956
3,0.019250,0.057810,0.971783
5,0.023290,0.065714,0.966844
15,0.024877,0.065238,0.965736
14,0.022828,0.078190,0.964297
6,0.025830,0.073429,0.963101
4,0.025859,0.075810,0.962525
13,0.026522,0.083048,0.960332
7,0.028398,0.078667,0.959911
8,0.027792,0.083048,0.959358


In [184]:
num_exp = 50
aggregates_20 = {g:0 for g in gestures}
for g in gestures:
    aggregates_20[g] = {}
    aggregates_20[g]['far'] = 0
    aggregates_20[g]['frr'] = 0
    aggregates_20[g]['accuracy'] = 0
for exp in tqdm(range(num_exp)):
    auth_set = set(np.random.choice(range(1, 44), size = 10, replace=False))
    for g in gestures:
        # aggregates_10_auth[g] = {}
        auths = []
        i = np.where(all_g==g)
        embeds = all_embeddings[i]
        labels = all_y[i]
        # print(len(labels), g)
        # for e in range(len(embeds)):
        #     embedding = np.array(embeds[e,:])
        #     auth_code = auth(embedding, auth_set, all_centroids, g, all_parts)
        #     auths.append(auth_code)
        auths = np.apply_along_axis(auth, axis=1, arr=embeds, auth_set=auth_set, centroids=all_centroids, gesture_val=g, parts=all_parts)
        actual_auth = [1 if x in auth_set else 0 for x in labels]

        aggregates_20[g]['far'] += far(actual_auth, auths)
        aggregates_20[g]['frr'] += frr(actual_auth, auths)
        aggregates_20[g]['accuracy'] += accuracy_score(actual_auth, auths)

for k in aggregates_10:
    aggregates_20[k]['far'] /= num_exp
    aggregates_20[k]['frr'] /= num_exp
    aggregates_20[k]['accuracy'] /= num_exp

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [06:43<00:00,  8.07s/it]


In [185]:
pd.DataFrame(aggregates_20).T.sort_values(by = "accuracy", ascending=False)

,far,frr,accuracy
2,0.014661,0.042000,0.978981
3,0.017807,0.065905,0.971008
5,0.021876,0.066095,0.967841
15,0.020808,0.072857,0.967087
14,0.022886,0.071524,0.965803
6,0.024589,0.074476,0.963810
4,0.024675,0.079238,0.962636
11,0.026032,0.086000,0.960022
8,0.026263,0.087333,0.959535
13,0.026724,0.089524,0.958671


In [260]:
df_list = [pd.DataFrame(aggregates_1_auth).T, pd.DataFrame(aggregates_10_auth).T, 
                          pd.DataFrame(aggregates_test).T, pd.DataFrame(aggregates_1).T, pd.DataFrame(aggregates_10).T]

In [263]:
joined = pd.DataFrame
for idx, df in enumerate(df_list):
    df = df.add_prefix(idx)
    if idx == 0:
        joined = df
        continue
    joined = pd.merge(joined, df, left_index=True, right_index=True)
    # duplicated_columns = joined.columns[joined.columns.duplicated()]
    # joined = joined.drop(columns=duplicated_columns)

In [264]:
joined.sort_values(by="4accuracy", ascending=False)

,0far,0frr,0accuracy,1far,1frr,1accuracy,2far,2frr,2accuracy,3far,3frr,3accuracy,4far,4frr,4accuracy
2,0.000798,0.017143,0.998772,0.004864,0.022095,0.990602,0.057460,0.073333,0.936190,0.006441,0.044000,0.989192,0.014372,0.038762,0.979956
3,0.001416,0.029524,0.997845,0.008265,0.032476,0.985363,0.051111,0.111905,0.924571,0.009323,0.059619,0.984828,0.019250,0.057810,0.971783
5,0.000927,0.047619,0.997845,0.011905,0.026381,0.984286,0.061270,0.095238,0.925143,0.010952,0.068952,0.982303,0.023290,0.065714,0.966844
15,0.000978,0.047619,0.997794,0.012993,0.035714,0.981028,0.058413,0.103333,0.923619,0.010652,0.072952,0.982104,0.024877,0.065238,0.965736
14,0.000952,0.035238,0.998145,0.007653,0.032000,0.985940,0.090794,0.143810,0.888000,0.011729,0.083810,0.979889,0.022828,0.078190,0.964297
6,0.001055,0.028571,0.998221,0.011667,0.027619,0.984135,0.033333,0.071905,0.951238,0.011529,0.082667,0.980199,0.025830,0.073429,0.963101
4,0.001210,0.022857,0.998221,0.010544,0.031905,0.983835,0.076825,0.107619,0.910857,0.011529,0.087048,0.979690,0.025859,0.075810,0.962525
13,0.001416,0.051429,0.997268,0.013810,0.039905,0.979323,0.054603,0.080476,0.935048,0.012932,0.092762,0.977785,0.026522,0.083048,0.960332
7,0.001364,0.030476,0.997870,0.009456,0.027333,0.985840,0.128571,0.190476,0.846667,0.012907,0.090476,0.978073,0.028398,0.078667,0.959911
8,0.001725,0.050476,0.996992,0.012653,0.052095,0.976967,0.087302,0.119048,0.900000,0.012757,0.093524,0.977852,0.027792,0.083048,0.959358
